In [1]:
#import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
tf.__version__

'2.3.2'

In [3]:
import tensorflow_hub as hub
import tensorflow as tf

In [5]:
elmo= hub.load("https://tfhub.dev/google/elmo/3")
#elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [6]:
elmo

In [7]:
x = ["Roasted ants are a popular snack in Columbia"]
x = tf.convert_to_tensor(x)
embeddings = elmo.signatures['default'](x)

In [8]:
embeddings['elmo']

<tf.Tensor: shape=(1, 8, 1024), dtype=float32, numpy=
array([[[ 0.11338113, -0.2238095 ,  0.2050071 , ..., -0.28580242,
          0.30716604,  0.38316193],
        [-0.3310474 ,  0.28298116,  0.12586498, ..., -0.03267071,
          0.7984375 ,  0.5640658 ],
        [ 0.18838716,  0.2805182 ,  0.01618184, ..., -0.44247535,
         -0.06120272,  0.08442858],
        ...,
        [-0.21390408, -0.16183913,  0.46044433, ...,  0.14843073,
          0.21630843,  0.58218354],
        [-0.32186395,  0.6526349 , -0.13358858, ..., -0.7157732 ,
         -0.4267435 ,  0.17269713],
        [ 0.05592338,  0.17106906, -0.1534959 , ...,  0.42320758,
         -0.2231236 ,  0.09431937]]], dtype=float32)>

In [23]:
#tf.compat.v1.enable_eager_execution()
#tf.compat.v1.disable_eager_execution()
def elmo_vectors(x):
    x = tf.constant(x, dtype=tf.string)
    #x=tf.convert_to_tensor(x)
    #embeddings = elmo(x, signature="default", as_dict=True)["elmo"]
    #embeddings=elmo.signatures['default'](x)
    #embeddings = elmo.signatures['default']
    #embeddings = embeddings['elmo']
    embeddings = elmo.signatures['default'](x)
    embeddings = embeddings['elmo']
    with tf.compat.v1.Session() as sess:
        sess.run( tf.compat.v1.global_variables_initializer())
        sess.run(tf.compat.v1.tables_initializer())
        Mean = np.mean(embeddings,1)
        Mean=tf.convert_to_tensor(Mean)
        return sess.run(Mean)


In [24]:
x = ["Roasted ants in Columbia"]
elmo_vectors(x)

array([[-0.08810478,  0.17094007,  0.02639696, ...,  0.09108515,
         0.08287467,  0.3194799 ]], dtype=float32)

In [27]:
dataset = ["Food is good and not too expensive. Serving is just right for adult. Ambient is nice too.",
           "Used to be good. Chicken soup was below average, bbq used to be good.",
           "Food was good, standouts were the spicy beef soup and seafood pancake! ",
           "Good office lunch or after work place to go to with a big group as they have a lot of private areas with large tables",
           "As a Korean person, it was very disappointing food quality and very pricey for what you get. I won't go back there again. ",
           "Not great quality food for the price. Average food at premium prices really.",
           "Fast service. Prices are reasonable and food is decent.",
           "Extremely long waiting time. Food is decent but definitely not worth the wait.",
           "Clean premises, tasty food. My family favourites are the clear Tom yum soup, stuffed chicken wings, chargrilled squid.",
           "really good and authentic Thai food! in particular, we loved their tom yup clear soup with sliced fish. it's so well balanced that we can have it just on its own. "
           ]

In [36]:
df=pd.DataFrame(dataset, columns=['name'])

In [61]:
df['Label']=ok
df

,name,Label
0,Food is good and not too expensive. Serving is...,1
1,Used to be good. Chicken soup was below averag...,0
2,"Food was good, standouts were the spicy beef s...",1
3,Good office lunch or after work place to go to...,0
4,"As a Korean person, it was very disappointing ...",1
5,Not great quality food for the price. Average ...,0
6,Fast service. Prices are reasonable and food i...,1
7,Extremely long waiting time. Food is decent bu...,1
8,"Clean premises, tasty food. My family favourit...",0
9,really good and authentic Thai food! in partic...,0


In [60]:
ok=[1,0,1,0,1,0,1,1,0,0]

In [62]:
train,test = train_test_split(df, test_size=0.20)

In [63]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]

In [64]:
elmo_train = [elmo_vectors(x['name']) for x in list_train]

In [65]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)

In [67]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['Label'],  
                                                  random_state=42, 
                                                  test_size=0.2)


In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)
preds_valid = lreg.predict(xvalid)
f1_score(yvalid, preds_valid)

0.0

In [69]:
df = pd.read_csv('preprocess_womens_clothing_reviews.csv')

In [70]:
X,y=df['Review Text no Stopwords'], df['Label']

In [71]:
dft = df[['Review Text no Stopwords','Label']].copy()

In [72]:
train,test = train_test_split(dft, test_size=0.20)

In [73]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [74]:
elmo_train = [elmo_vectors(x['Review Text no Stopwords']) for x in list_train]
elmo_test = [elmo_vectors(x['Review Text no Stopwords']) for x in list_test]

In [75]:
elmo_train

[array([[ 0.01836874, -0.0529776 , -0.03961765, ...,  0.03003482,
          0.16536346,  0.10915444],
        [ 0.082944  , -0.06487523,  0.12519158, ...,  0.05115635,
          0.3651384 ,  0.17732887],
        [-0.03333084, -0.0199105 ,  0.10287441, ...,  0.05072099,
         -0.03066421,  0.16569649],
        ...,
        [ 0.08995169,  0.00590855,  0.07604367, ...,  0.03713353,
          0.02664421,  0.08795018],
        [-0.01347574, -0.01129613,  0.04081788, ...,  0.07921408,
          0.00405115,  0.0084268 ],
        [ 0.10633784, -0.02286263,  0.08378907, ...,  0.0379558 ,
         -0.16976528,  0.30102795]], dtype=float32),
 array([[ 0.15098317,  0.10501827, -0.03181308, ...,  0.0486211 ,
          0.00071519,  0.15982139],
        [ 0.03718632, -0.10723287,  0.14621766, ..., -0.00484867,
         -0.11978117,  0.17248535],
        [ 0.2864605 ,  0.12798   ,  0.14337043, ...,  0.06698939,
         -0.21541394,  0.11894976],
        ...,
        [ 0.22222598, -0.00613662,  0.0

In [76]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [77]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['Label'],  
                                                  random_state=42, 
                                                  test_size=0.2)


In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)
preds_valid = lreg.predict(xvalid)
f1_score(yvalid, preds_valid)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.910241066849034

In [ ]:
train.loc[12124]